# Hugging Face Large Model Inference

This notebook demonstrates how to deploy Hugging Face transformer models using Hugging Face Text Generation Inference (TGI) Deep Learning Container on Amazon SageMaker.

TGI is an open source, high performance inference library that can be used to deploy large language models from Hugging Face’s repository in minutes. The library includes advanced functionality like model parallelism and dynamic batching to simplify production inference with large language models like flan-t5-xxl, LLaMa, StableLM, and GPT-NeoX. 

## Setup

### Install the SageMaker Python SDK

First, make sure that the latest version of SageMaker SDK is installed.

In [2]:
%pip install "sagemaker>=2.163.0"

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.3
    Uninstalling urllib3-2.0.3:
      Successfully uninstalled urllib3-2.0.3
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.6.0
    Uninstalling importlib-metadata-6.6.0:
      Successfully uninstalled importlib-metadata-6.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
spyder 4.0.1 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.0.1 requires pyqtwebengine<5.13; python_version >= "3", which is not 

### Setup account and role

Then, we import the SageMaker python SDK and instantiate a `sagemaker_session` which we use to determine the current region and execution role.

In [3]:
import sagemaker
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
import time

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

## Retrieve the LLM Image URI

We use the helper function `get_huggingface_llm_image_uri()` to generate the appropriate image URI for the Hugging Face Large Language Model (LLM) inference.

The function takes a required parameter `backend` and several optional parameters. The `backend` specifies the type of backend to use for the model, the values can be "lmi" and "huggingface". The "lmi" stands for SageMaker LMI inference backend, and "huggingface" refers to using Hugging Face TGI inference backend.

In [4]:
image_uri = get_huggingface_llm_image_uri(
  backend="huggingface", # or lmi
  region=region
)

## Create the Hugging Face Model

Next we configure the `model` object by specifying a unique name, the `image_uri` for the managed TGI container, and the execution role for the endpoint. Additionally, we specify a number of environment variables including the `HF_MODEL_ID` which corresponds to the model from the HuggingFace Hub that will be deployed, and the `HF_TASK` which configures the inference task to be performed by the model.

You should also define `SM_NUM_GPUS`, which specifies the tensor parallelism degree of the model. Tensor parallelism can be used to split the model across multiple GPUs, which is necessary when working with LLMs that are too big for a single GPU. Here, you should set `SM_NUM_GPUS` to the number of available GPUs on your selected instance type.  For example, in this tutorial, we set `SM_NUM_GPUS` to 4 because our selected instance type `ml.g5.12xlarge` has 4 available GPUs.

Additionally, we could reduce the memory footprint of the model by setting the `HF_MODEL_QUANTIZE` environment variable to true.

In [25]:
model_name = "falcon-40b-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

hub = {
    'HF_MODEL_ID':'tiiuae/falcon-40b-instruct',
    'HF_TASK':'text-generation',
    'SM_NUM_GPUS':'4'
}

model = HuggingFaceModel(
    name=model_name,
    env=hub,
    role=role,
    image_uri=image_uri
)

## Creating a SageMaker Endpoint

Next we deploy the model by invoking the `deploy()` function.

To efficiently deploy and run large language models, it is important to choose an appropriate instance type that can handle the computational requirements. Here we use an `ml.g5.12xlarge` instance which come with 4 NVIDIA A10 GPUs. By setting the `SM_NUM_GPUS` environment variable to 4 in the last code block, we indicate that this model should be sharded across all 4 GPU devices.

Please refer to the [guide](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-choosing-instance-types.html) provided by AWS SageMaker on large model inference instance type selection.

In [6]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.12xlarge",
    endpoint_name=model_name
)

---------------!

## Running Inference

Once the endpoint is up and running, we can evaluate the model using the `predict()` function.

In [24]:
%%time

input_data = {
  "inputs": "What is Love?",
  "parameters": {
    "do_sample": True,
    "max_new_tokens": 150,
    "temperature": 0.7,
    "watermark": True
  }
}

predictor.predict(input_data)

CPU times: user 5.15 ms, sys: 0 ns, total: 5.15 ms
Wall time: 3.36 s


[{'generated_text': 'What is Love?\nLove is a powerful emotion that encompasses a range of strong and positive feelings. It can refer to the affection felt by family members, friends, and romantic partners. Love can inspire kindness, compassion, and selflessness, and can motivate individuals to overcome challenges and make meaningful contributions to their communities and the world at large.'}]

## Cleaning Up

After you've finished using the endpoint, it's important to delete it to avoid incurring unnecessary costs.

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

## Conclusion

In this tutorial, we used a TGI container to deploy Falcon-40B using 4 GPUs on a SageMaker `ml.g5.12xlarge` instance. With Hugging Face's Text Generation Inference and SageMaker Hosting, you can easily host large language models like GPT-NeoX, flan-t5-xxl, and LLaMa.